<a href="https://colab.research.google.com/github/kavyajeetbora/foursquare_ai/blob/master/12_market_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import zipfile
import os

url = "https://alltheplaces-data.openaddresses.io/runs/2025-12-06-13-33-06/output.zip"
zip_path = "output.zip"
dir_path = "atp_data"

# Remove old directory if it exists (to ensure clean extract)
if os.path.exists(dir_path):
    !rm -rf {dir_path}

# Download with progress bar (quiet in terms of no verbose output, but shows progress)
print("Downloading the large zip file (~1-2 GB compressed)... This may take several minutes.")
urllib.request.urlretrieve(url, zip_path)

# Get file size
size_gb = os.path.getsize(zip_path) / (1024**3)
print(f"Download completed: {size_gb:.2f} GB")

# Unzip quietly
print("Extracting files quietly... This will also take a few minutes.")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dir_path)

# Count extracted GeoJSON files
geojson_files = [f for f in os.listdir(dir_path) if f.endswith('.geojson')]
print(f"Extraction complete: {len(geojson_files)} GeoJSON files extracted to '{dir_path}/'")

In [ ]:
%%shell
wget https://github.com/kavyajeetbora/foursquare_ai/blob/22c6962e1a6d1f95239e9c896e21d60824282dbd/india.geojson -O india_boundary.geojson

In [ ]:
import geopandas as gpd

india_boundary = gpd.read_file('india_boundary.geojson', driver)

In [ ]:
import duckdb

# Use your existing connection or create new
con = duckdb.connect()

# Ensure spatial is loaded (safe to run multiple times)
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# Now create the table using ST_Read_Multi with wildcard
# This will read ALL GeoJSON files in the directory and union them
df = con.execute(f"""
    SELECT *
    FROM ST_Read({files})
""").df()

con.close()
df.head()

In [ ]:
from glob import glob

files = glob('atp_data/output/*_in.geojson')
print(len(files))